In [ ]:
using Images
using PlotlyJS
using Colors, ColorVectorSpace, FixedPointNumbers, Statistics
using Interact
using Distances, Plots
using BenchmarkTools

In [ ]:
img26 = load("dataset/rgb/FISH_Brachy/Image0026-rgb.tif");

In [ ]:
tif_red=load("dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=r.tif");
tif_green=load("dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=g.tif");
tif_blue=load("dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=b.tif");

In [ ]:
function get_coords(img)
    y_size, x_size, z_size = size(img)

    z_list = zeros(0)

    for i in 1:z_size
        append!(z_list, fill(i ,y_size * x_size))
    end

    x_list = zeros(0)
    for i in 1:x_size
        append!(x_list, fill(i ,y_size))
    end  

    append!(x_list, repeat(x_list,z_size-1))

    y_list = collect(1:y_size)
    append!(y_list, repeat(y_list,z_size * x_size-1));
    
    return hcat(y_list,x_list,z_list)
end

Połączenie współrzędnych z kolorami i rozmiarem dla img26

In [ ]:
coords_matrix = get_coords(img26);
img26_flat = vec(img26)
img26_flat = RGB{Float64}.(img26_flat)
ones_array = ones(length(img26_flat))
img26_arr=hcat(coords_matrix, red.(img26_flat), green.(img26_flat), blue.(img26_flat), ones_array);

Połączenie współrzędnych z kolorami i rozmiarem dla exp392

In [ ]:
coords_matrix = get_coords(tif_red)

# flatten 3d array into 1d array
tif_red_flat = vec(tif_red)
tif_green_flat = vec(tif_green)
tif_blue_flat = vec(tif_blue)


# tif_flat = RGB{Float64}.(tif_red_flat*4,tif_green_flat*4,tif_blue_flat*4)

# # # creating column of ones with size as number of pixels
ones_array = ones(length(tif_red_flat))

# # # connecting 1d arrays like columns into 2d array
# # #img26_arr=hcat(y_list,x_list,z_list, img26_flat, ones_array);
tif_arr=hcat(coords_matrix, Float64.(tif_red_flat*4), Float64.(tif_green_flat*4), Float64.(tif_blue_flat*4), ones_array);

### Usuwanie czarnego tła

In [ ]:
function clear_black_bg(tif_arr, threshold)
    tif_arr = tif_arr[vec(mapslices(col -> (col[4] .> threshold) | (col[5] .> threshold) | (col[6] .> threshold), 
            tif_arr, dims = 2)), :]
    return tif_arr
end

In [ ]:
img26_arr_clr=clear_black_bg(img26_arr,0.393);

In [ ]:
tif_arr_clr=clear_black_bg(tif_arr,0.5);

### funkcje pomocnicze

In [ ]:
function get_centroid(array_2d)
    length = size(array_2d, 1)
    sum_y = sum(array_2d[:,1])
    sum_x = sum(array_2d[:,2])
    sum_z = sum(array_2d[:,3])
    color_mean=mean(array_2d[:,4])
    
    return hcat(sum_y/length, sum_x/length, sum_z/length, mean(array_2d[:,4]), mean(array_2d[:,5]), mean(array_2d[:,6]), length)
end

In [ ]:
function get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
    
    n = tif_arr[vec(mapslices(col -> (col[1] .<= pixel[1]+radius) & (col[1] .>= pixel[1]-radius)
                                    & (col[2] .<= pixel[2]+radius) & (col[2] .>= pixel[2]-radius)
                                    & (col[3] .<= pixel[3]+zradius) & (col[3] .>= pixel[3]-zradius)
                                    & (col[4] .<= pixel[4]+color_diff) & (col[4] .>= pixel[4]-color_diff)
                                    & (col[5] .<= pixel[5]+color_diff) & (col[5] .>= pixel[5]-color_diff)
                                    & (col[6] .<= pixel[6]+color_diff) & (col[6] .>= pixel[6]-color_diff), 
                                    tif_arr, dims = 2)), :]
    return n
end

### Centroid Method, do array z odosobnionymi kolorami

In [ ]:
function centroid_method(tif_arr,radius,zradius,color_diff)
    # defining empty matrix where centroids will be stored
    centroids = Array{Float64}(undef, 0, 7)

    # looping thorugh tif_arrlicated array until it's empty
    while size(tif_arr, 1)>0

        # assign starting point to random pixel
        pixel = tif_arr[rand(1:end), :] 

        n=get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
        
        # calculating centroid vector from array of neighbours
        centroid = get_centroid(n)

        # adding centroid to centroids matrix
        centroids = [centroids; centroid]

        # deleting neighbours from tif_arrlicated array
        tif_arr=tif_arr[vec(mapslices(row -> !(row in eachrow(n)), tif_arr, dims = 2)),:]

    end
    
    return centroids
end

### medoid method

In [ ]:
function get_medoid(n)
    distMatrix=pairwise(Euclidean(),n[:,3:6],dims=1)
    medoid=n[argmin(sum(distMatrix,dims=2))[1],:]
    medoid=reshape(medoid, 1, length(medoid))
    medoid[7]=size(n,1)
    
    return medoid
end

In [ ]:
function medoid_method(tif_arr,radius,zradius,color_diff)
# defining empty matrix where centroids will be stored
    medoids = Array{Float64}(undef, 0, 7)

    # looping thorugh tif_arrlicated array until it's empty
    while size(tif_arr, 1)>0

        # assign starting point to random pixel
        pixel = tif_arr[rand(1:end), :] 

        # creating matrix of neighbours
        n=get_neighbours(tif_arr, pixel, radius, zradius, color_diff)

        # adding centroid to centroids matrix

        medoid=get_medoid(n)
        medoids=[medoids;medoid]
        # deleting neighbours from tif_arrlicated array
        tif_arr=tif_arr[vec(mapslices(row -> !(row in eachrow(n)), tif_arr, dims = 2)),:]

    end
    
    return medoids
end

In [ ]:
function plotly_vis(matrix, smin, smax)
    color_arr= RGB{Float64}.(matrix[:,4],matrix[:,5],matrix[:,6])
    return PlotlyJS.plot(PlotlyJS.scatter3d(
            ;x=matrix[:,2], 
            y=matrix[:,1], 
            z=matrix[:,3],
            text=matrix[:,7],
            customdata=color_arr,
#             color=matrix[:,4], # opcja jeśli nie rozdielono kolorów
            type="scatter3d",
            mode="markers",
            marker=attr(
                sizemode="area",
                size=matrix[:,7],
                color=color_arr,
                sizeref=(maximum(matrix[:,7])) / (smax ^ 2),
                sizemin=smin,
                opacity=1,
                line=attr(width=0)
            ),

            hovertemplate="<br>x:%{x}<br>y:%{y}<br>z:%{z}<br>color: %{customdata}<br>size: %{text}"
        )
    )
end

In [ ]:
function tif_reduction_widget(tif_arr)
    radius = slider(10:20, label = "radius")
    zradius = slider(1:5, label = "zradius")
    color_diff = slider(0:0.01:.3, label = "color_diff")
    smin = slider(0:20, label = "smin")
    smax = slider(40:60, label = "smax")
    method = dropdown(OrderedDict("centroid method" => "centroid_method", "medoid method" => "medoid_method"),label="method")
    update = button("compute")
    output= Observable{Any}((undef, 0, 0))
    plt=Plots.plot()
    Interact.@map (&update; 
        if  method[] == "medoid_method"
            output[] = medoid_method(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "centroid_method"
            output[] = centroid_method(tif_arr,radius[],zradius[],color_diff[])
        end
    )
    plt = Interact.@map plotly_vis(&output,&smin,&smax)
    wdg = Widget(["radius" => radius, "zradius" => zradius, 
            "color_diff" => color_diff, "update" => update, 
            "smin" => smin, "smax" => smax, 
            "method"=>method], output = output);
    @layout! wdg hbox(plt, vbox(:radius, :zradius, :color_diff, pad(1em,:method), :update, :smin, :smax))

end

In [ ]:
tif_reduction_widget(img26_arr_clr)

In [ ]:
centroids=centroid_method(img26_arr,15,2,0.0393);

In [ ]:
plotly_vis(centroids)

### Pomiary

In [ ]:
@time centroid_method(img26_arr,10,1,0.0393);

In [ ]:
@btime centroid_method(img26_arr,10,1,0.0393)

In [ ]:
@benchmark centroid_method($img26_arr,10,1,0.0393)

In [ ]:
@benchmark centroid_method($img26_arr,5,1,0.0393)

In [ ]:
@benchmark medoid_method($img26_arr,10,1,0.0393)

In [ ]:
@benchmark get_centroid($img26_arr)